In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from survival import DecayMIRModel
from survival import MIRModel
from survival import AgeSurvivalModel
from survival import LogFormMIRModel

ImportError: cannot import name 'AgeSurvivalModel'

## INPUT DATA

### [GBD 2019 MIRs with GBD 2019 background mortality]

In [2]:
#PROCESS INPUTS
df = pd.read_csv('/home/j/temp/fed1/2019_mirs_with_background_mortality.csv')

#remove NAs and MIR >1
df = df.dropna()
df = df[df['mi_ratio']<=1]

df.head()

,Unnamed: 0,location_id,year_id,sex_id,age_group_id,acause,mi_ratio,life_table_parameter_id,background_mortality
0,1,6,1980,1,1,neo_bladder,0.296086,3.0,0.076672
1,2,6,1980,1,1,neo_brain,0.670154,3.0,0.076672
2,3,6,1980,1,1,neo_breast,0.486486,3.0,0.076672
3,4,6,1980,1,1,neo_colorectal,0.560212,3.0,0.076672
4,5,6,1980,1,1,neo_esophageal,0.701150,3.0,0.076672


In [5]:
df = df[df['location_id']==102]

In [7]:
df.head()

,Unnamed: 0,location_id,year_id,sex_id,age_group_id,acause,mi_ratio,life_table_parameter_id,background_mortality,excess_mortality_og_mir_model,abs_survival_rate_og_mir_model,rel_survival_rate_og_mir_model
6605760,6605761,102,1980,1,1,neo_bladder,0.076585,3.0,0.016757,0.008572,0.879610,0.957161
6605761,6605762,102,1980,1,1,neo_brain,0.346066,3.0,0.016757,0.045421,0.725441,0.789400
6605762,6605763,102,1980,1,1,neo_breast,0.078844,3.0,0.016757,0.008835,0.878423,0.955870
6605763,6605764,102,1980,1,1,neo_colorectal,0.267017,3.0,0.016757,0.033277,0.773644,0.841853
6605764,6605765,102,1980,1,1,neo_esophageal,0.638391,3.0,0.016757,0.108318,0.512690,0.557891


## I. Base MIR -> Survival Model:

#### [M/I = (1-P_s^n)(P_c)/(P_c+P_o)]

In [6]:
#PARAMETERS
num_years = 5
disease_pd = 10

#RUN BASE MODEL
model = MIRModel(df['mi_ratio'],
                 df['background_mortality'],
                 disease_period=disease_pd)
model.compute_excess_mortality()
survival_rate = model.get_survival_rate(num_years=num_years)

# SAVE IN ORIGINAL DATA FRAME
df['excess_mortality_og_mir_model'] = model.excess_mortality
df['abs_survival_rate_og_mir_model'] = survival_rate['abs']
df['rel_survival_rate_og_mir_model'] = survival_rate['rel']

/ihme/homes/fed1/.conda/envs/survival_env/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/ihme/homes/fed1/.conda/envs/survival_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/ihme/homes/fed1/.conda/envs/survival_env/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

## II. Age Group Extension Model:

In [ ]:
#PARAMETERS
num_years = 5

# RUN AGE EXTENSION MODEL
age_model = AgeSurvivalModel(df)
age_model.compute_P_s()

# Compute the n year survival in model
age_model.compute_n_year_survival(num_years=num_years)

# Compile outputs of model into dataframe
outputs = age_model.combine_outputs()
df.head()

# SAVE IN ORIGINAL DATA FRAME
?

## III. Exponential decay of probability of death model:

#### [log(P_c) = mx + b]

In [ ]:
# PARAMETERS
num_years = 5
df['disease_period'] = 10
df['slope'] = .21

# RUN DECAY MODEL
decay_model = DecayMIRModel(df['mi_ratio'],
                 df['other_mortality'],
                 df['disease_period'],
                 df['slope'])
decay_model.compute_base_excess_mortality()
survival_rate = decay_model.get_survival_rate(num_years=num_years)

# SAVE IN ORIGINAL DATA FRAME
df['base_excess_mortality_decay_model'] = decay_model.base_excess_mortality
df['abs_survival_rate_decay_model'] = survival_rate['abs']
df['rel_survival_rate_decay_model'] = survival_rate['rel']

## IV. Three parameter decay model:

#### [log(P_c) = b*log(1+exp(-(x-a)))+c]

In [ ]:
# PARAMETERS: all cause parameters used for testing
parameters = pd.read_csv('/homes/fed1/Survival_Franny/three_parameter_log_form_values_SEER_5_4_2021.csv')
df['a'] = parameters[parameters['cause_name']=="All Sites"]
df['b'] = parameters[parameters['cause_name']=="All Sites"]

# RUN THREE PARAMETER LOG MODEL


# SAVE IN ORIGINAL DATA FRAME
?

## Comparison to Current GBD Methods

#### [Regress on 1-MIR]

In [ ]:
#READ IN AND APPEND OLD SURVIVAL PREDICTIONS TO COMPARE



## Prediction Scatters

In [ ]:
for cancer in df.acause.unique():
        
        data = df[df['acause']==cancer]

        # PLOT
        output = function(res_lsq.x, t)
        plt.plot(t, y, 'o', markersize=4, label='data')
        plt.plot(t, output, label='fitted model')
        plt.xlabel("t")
        plt.ylabel("log_P_c")
        plt.legend(loc='lower right')
        plt.title(cancer)#+": a ="+str(res_lsq.x[0])+" b ="+str(res_lsq.x[1])+" c ="+str(res_lsq.x[2]))
        plt.show()
        
        # SAVE
        row = pd.DataFrame(data={'cause_name': [cancer],
              'a': [res_lsq.x[0]],
              'b': [res_lsq.x[1]],
              'c': [res_lsq.x[2]]})
        print(row)
        parameters = parameters.append(row, ignore_index = True)

In [ ]:
df.to_csv('/home/j/temp/fed1/comparing_all_survival_methods_6_1_2021.csv')